In [24]:
import whisperx
import os
device = "cuda" 
batch_size = 16 # reduce if low on GPU mem
compute_type = "float16" # change to "int8" if low on GPU mem (may reduce accuracy)

# 1. Transcribe with original whisper (batched)
model = whisperx.load_model("i4ds/whisper4sg-srg-full-mc-de-sg-corpus", device, compute_type=compute_type, language='de')


No language specified, language will be first be detected for each audio file (increases inference time).


Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.2.2. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../.cache/torch/whisperx-vad-segmentation.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.2.2+cu121. Bad things might happen unless you revert torch to 1.x.


In [25]:
audio = whisperx.load_audio("Schönleber_1.mp3")
result = model.transcribe(audio, batch_size=batch_size, language="de")
print(result["segments"]) # before alignment

[{'text': ' Ich rufe Frau Schönleber an. Können Sie mir kurz erzählen, warum Sie heute bei mir anrufen? Genau, ich rufe an, weil ich nicht mehr zum Arzt komme.', 'start': 1.459, 'end': 19.155}, {'text': ' Ich habe das Problem von mir hergeschoben. Seit acht Wochen habe ich einen Husten. Der geht und geht und geht einfach nicht weg. Normalerweise, wenn ich einen Husten hatte, merke ich, dass irgendwann ein Zeug rauskommt. Dann geht es auch weg. Aber jetzt kommt nichts mehr. Ich habe keinen Auswurf. Am Morgen, am Abend, über den Mittag, in der Nacht ... Es ist einfach konstant.', 'start': 20.128, 'end': 49.684}, {'text': ' Und es ist auch egal, was ich mache. Ob ich liege, stehe, sitze. Es ist einfach eine Konstante in meinem Leben. Und jetzt musste ich mich einfach melden. Ja. Hatten Sie das Gefühl, als es angefangen hat, dass Sie mal krank waren? Hatten Sie sonst noch eine anhängige, laufende Nase? Halsweh oder so etwas? Also vor acht Wochen wäre das, oder? Ja, genau als es angefangen 

In [26]:
# 2. Align whisper output
model_a, metadata = whisperx.load_align_model(language_code=result["language"], device=device)
result = whisperx.align(result["segments"], model_a, metadata, audio, device, return_char_alignments=False)

print(result["segments"]) # after alignment


[{'start': 3.361, 'end': 6.784, 'text': ' Ich rufe Frau Schönleber an.', 'words': [{'word': 'Ich', 'start': 3.361, 'end': 3.881, 'score': 0.632}, {'word': 'rufe', 'start': 4.001, 'end': 5.463, 'score': 0.81}, {'word': 'Frau', 'start': 5.503, 'end': 5.883, 'score': 0.824}, {'word': 'Schönleber', 'start': 5.923, 'end': 6.644, 'score': 0.781}, {'word': 'an.', 'start': 6.684, 'end': 6.784, 'score': 0.56}]}, {'start': 6.904, 'end': 13.67, 'text': 'Können Sie mir kurz erzählen, warum Sie heute bei mir anrufen?', 'words': [{'word': 'Können', 'start': 6.904, 'end': 8.505, 'score': 0.517}, {'word': 'Sie', 'start': 8.525, 'end': 8.625, 'score': 0.534}, {'word': 'mir', 'start': 8.645, 'end': 8.746, 'score': 0.419}, {'word': 'kurz', 'start': 8.766, 'end': 8.946, 'score': 0.524}, {'word': 'erzählen,', 'start': 9.006, 'end': 9.486, 'score': 0.596}, {'word': 'warum', 'start': 9.506, 'end': 9.726, 'score': 0.141}, {'word': 'Sie', 'start': 9.746, 'end': 9.867, 'score': 0.317}, {'word': 'heute', 'start'

In [27]:
import pysubs2

subs = pysubs2.load_from_whisper(result)
subs.save("Schönleber_sota_whisperx.srt")